# Housing Rental Analysis for San Francisco

In [ ]:
import pandas as pd
import hvplot.pandas
from pathlib import Path

sfo_data_df = pd.read_csv(Path("Resources/sfo_neighborhoods_census_data.csv"))
display(sfo_data_df)

---

## Calculate and Plot the Housing Units per Year

In [ ]:
housing_units_by_year = sfo_data_df.groupby(by='year').mean()
display(housing_units_by_year)

In [ ]:
from bokeh.models.formatters import NumeralTickFormatter

sfo_data_df.hvplot.bar(
    x='year', y='housing_units',
    yformatter=NumeralTickFormatter(format='0,0a'),
    xlabel='Year', ylabel='Housing units',
    title='Housing units by year, 2010-16',
    fontsize=14,
    height=400, width=600,
    hover=False,
    ylim=(370000,None),
)

As shown by the graph, housing unit numbers steadily increased.

---

## Calculate and Plot the Average Sale Prices per Square Foot

In [ ]:
prices_square_foot_by_year = sfo_data_df.groupby('year').mean()
display(prices_square_foot_by_year)

**Question:** What is the lowest gross rent reported for the years included in the DataFrame?

**Answer:** $1,239

In [ ]:
prices_square_foot_by_year = prices_square_foot_by_year[['sale_price_sqr_foot', 'gross_rent']]
display(prices_square_foot_by_year)

In [ ]:
prices_square_foot_by_year.hvplot.line(
    xlabel='Year', ylabel='Gross Rent, Sale Price / SqFt',
    title='Avg. Gross Rent and Sale Price / SqFt ($), 2010-16',
    height=600,
).opts(legend_position='top_left', toolbar=None)

**Question:** Did any year experience a drop in the average sale price per square foot compared to the previous year?

**Answer:** Yes, 2010.

**Question:** If so, did the gross rent increase or decrease during that year?

**Answer:** Increase.

---

## Compare the Average Sale Prices by Neighborhood

In [ ]:
prices_by_year_by_neighborhood = sfo_data_df.groupby(['neighborhood', 'year']).mean()
display(prices_by_year_by_neighborhood)

In [ ]:
prices_by_year_by_neighborhood = prices_by_year_by_neighborhood[['sale_price_sqr_foot', 'gross_rent']]
display(prices_by_year_by_neighborhood)

In [ ]:
prices_by_year_by_neighborhood.hvplot.line(
    x='year', xlabel='Year', ylabel='USD',
    yformatter=NumeralTickFormatter(format='$0,0a'),
    title='Gross rent and sale price per sq. ft., by neighborhood',
    groupby='neighborhood', 
    ylim=(0,None),
    height=700, width=620,
    dynamic=False,
).opts(legend_position='top_left', toolbar=None)

**Question:** For the Anza Vista neighborhood, is the average sale price per square foot for 2016 more or less than the price that’s listed for 2012? 

**Answer:** Less, 88.402 in 2016 compared with 344.491 (USD).

---

## Build an Interactive Neighborhood Map

In [ ]:
# Load neighborhoods coordinates data
neighborhood_locations_df = pd.read_csv('Resources/neighborhoods_coordinates.csv', index_col='Neighborhood')
display(neighborhood_locations_df)

In [ ]:
# Calculate the mean values for each neighborhood
all_neighborhood_info_df = sfo_data_df.groupby('neighborhood').mean()
display(all_neighborhood_info_df)

In [ ]:
all_neighborhoods_df = pd.concat(
    [neighborhood_locations_df, all_neighborhood_info_df], 
    axis="columns",
    sort=False
)

display(all_neighborhoods_df)

In [ ]:
# Call the dropna function to remove any neighborhoods that do not have data
all_neighborhoods_df = all_neighborhoods_df.reset_index().dropna()
all_neighborhoods_df = all_neighborhoods_df.rename(columns={"index": "Neighborhood"})
display(all_neighborhoods_df)

In [ ]:
all_neighborhoods_df.hvplot.points(
    'Lon', 'Lat',
    geo=True, tiles='EsriReference',
    size='sale_price_sqr_foot', color='gross_rent', 
    title='Interactive neighborhood map',
    xlabel='Longitude', ylabel='Latitude', 
    frame_width=700, frame_height=500, 
)

**Question:** Which neighborhood has the highest gross rent, and which has the highest sale price per square foot?

**Answer:** Westwood Park had the highest gross rent and Union Square District had the highest sale price per square foot.

## Data Story

**Question:**  How does the trend in rental income growth compare to the trend in sales prices? Does this same trend hold true for all the neighborhoods across San Francisco?

**Answer:** Rental income growth significantly outpaced sale price per square foot, on average. In general, yes, the trend held, although notably FiDi South had a decrease in sale price per square foot.

**Question:** What insights can you share with your company about the potential one-click, buy-and-rent strategy that they're pursuing? Do neighborhoods exist that you would suggest for investment, and why?

**Answer:** I would not be comfortable suggesting an investment based on the analysis presented here. If housing were a simple commodity with high liquidity, then the trend of increasing sale price in proportion to rent would suggest that providing a low-friction purchase platform could be profitable.